# Create a linear model in keras

In [2]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [3]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

## Create input and output for the model

In [3]:
x = random((30, 2))

In [4]:
y = np.dot(x, [2., 3.]) + 1

In [5]:
x[:5]

array([[ 0.511 ,  0.2324],
       [ 0.2859,  0.572 ],
       [ 0.2922,  0.4626],
       [ 0.7202,  0.3237],
       [ 0.1729,  0.3892]])

In [6]:
y[:5]

array([ 2.719 ,  3.2878,  2.9723,  3.4115,  2.5133])

In [8]:
lm = Sequential([Dense(1, input_shape=(2,))])
lm.compile(optimizer=SGD(lr=0.1), loss='mse')

### Check the preexisting loss function for x, y

In [9]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


9.0879507064819336

### Fit the model to minimise the loss function for x and y using gradient descent

In [10]:
lm.fit(x, y, nb_epoch=5, batch_size=1)

Epoch 1/5
30/30 [==============================] - 0s - loss: 0.7620     
Epoch 2/5
30/30 [==============================] - 0s - loss: 0.1227     
Epoch 3/5
30/30 [==============================] - 0s - loss: 0.0664     
Epoch 4/5
30/30 [==============================] - 0s - loss: 0.0384     
Epoch 5/5
30/30 [==============================] - 0s - loss: 0.0212     


In [11]:
lm.get_weights()

[array([[ 1.6424],
        [ 2.7709]], dtype=float32), array([ 1.3385], dtype=float32)]

In [12]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


0.01724732294678688

## Creating a linear model on predictions

First we like always create our untrained VGG model and get the data from the training and validation folders. Before that we set the paths obviously.

In [5]:
#path='data/dogscats/'
path='data/dogscats/sample/'
model_path=path + 'model/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [7]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [8]:
#Setting the batch size to be large because we are crazy like that
batch_size=100

In [9]:
batches=get_batches(path+'train/', shuffle=False, batch_size=1)
val_batches=get_batches(path+'valid/', shuffle=False, batch_size=1)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


Using functions that concatenate all the batches so that we can save the entire array and dont have to proces the images everytime

In [10]:
train_data = get_data(path+"train/")
val_data = get_data(path+'valid/')

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


Defining functions to save the arrays. We will be using the python library bcolz for that

In [12]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [13]:
val_data.shape

(8, 3, 224, 224)

In [14]:
save_array(model_path+'train_data_sample.bc', train_data)
save_array(model_path+'val_data_sample.bc', val_data)

In [15]:
val_data=load_array(model_path+'val_data_sample.bc')
train_data=load_array(model_path+'train_data_sample.bc')

In [16]:
val_data.shape

(8, 3, 224, 224)